In [ ]:
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
from marker_analyser.classes import OscillationCollection
from marker_analyser.configuration import FitConfig, ParamConfig
from pydantic import BaseModel, model_validator, ValidationError

from lumicks import pylake
from typing import Self

In [ ]:
# Load the data
with open("./experimenting_data/marina_data_loaded_oscillations.pkl", "rb") as file:
    oscillations: OscillationCollection = pickle.load(file)
    assert type(oscillations) == OscillationCollection

# get only the first 10 oscillations for testing
first_10_oscillation_ids = list(oscillations.oscillations.keys())[:10]
oscillations = OscillationCollection(
    oscillations={osc_id: oscillations.oscillations[osc_id] for osc_id in first_10_oscillation_ids}
)

print(f"Loaded {len(oscillations.oscillations)} oscillations for testing.")

In [ ]:
fit_config = FitConfig(
    params_config={
        "Lp": ParamConfig(lower_bound=0, upper_bound=None, initial_value=5, global_param=False, fixed=False),
        "Lc": ParamConfig(lower_bound=0, initial_value=1, global_param=True, fixed=False),
    },
    auto_calculate_and_fix_f_offset=True,
    f_offset_auto_detect_distance_range_um=(10, 12),
    model_name="fit",
    segment="both",
)

oscillations.fit_global_model_to_all(fit_config=fit_config)

# oscillations.plot_all(random_colours=True)

file_results_path = Path("./experimenting_data/global_fit_results.csv")
oscillations.global_fit_save_parameters_to_csv_file(file_path=file_results_path)